In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg, scipy.integrate
import do
from importlib import reload
from IPython.display import display, clear_output
from quantum_systems import ODQD, GeneralOrbitalSystem

In [2]:
### PARAMETERS ###
l = 10 # Number of eigenstates of the HO potential --> we use these functions to generate the single particle WF
grid_length = 10  # The width of the one-dimensional grid
num_grid_points = 201  # The number of discretized points on the grid.
# More points give better results for the single-particle basis at the cost of slower setup.
alpha = 1  # The strength of the Coulomb interaction 
a = 0.25  # The shielding parameter in the Coulomb interaction potential
Omega = 0.25  # The frequency of the harmonic oscillator trap
omega = 8*Omega # frequency of the laser field
epsilon0 = 1.0 # amplitude of the laser field
potential=ODQD.HOPotential(Omega)
nparticles = 2

In [85]:
reload(do)
odho = ODQD(l, grid_length, num_grid_points, a=a, alpha=alpha, potential=potential)
system = GeneralOrbitalSystem(n=nparticles, basis_set=odho, anti_symmetrize=True)
epsilon, C0 = do.solve_TIHF(system, nparticles, tolerance=1e-7, max_iter=1000, print_on=False)
system.change_basis(C0)
print(system.compute_reference_energy())

(1.1795712853092604+0j)


In [86]:
reload(do)
odho = ODQD(l, grid_length, num_grid_points, a=a, alpha=alpha, potential=potential)
system = GeneralOrbitalSystem(n=nparticles, basis_set=odho, anti_symmetrize=True)
epsilon, C0 = do.solve_TIHF(system, nparticles, tolerance=1e-7, max_iter=1000, print_on=False)
do.change_basis(system, C0)
print(system.compute_reference_energy())

(1.1795712853092604+0j)


000e+00+0.j],
         [-7.08797170e-18+0.j, -2.77555756e-17+0.j,
           1.58818678e-22+0.j, ..., -4.44522891e-18+0.j,
           5.29395592e-23+0.j, -6.97127537e-24+0.j]],

        [[ 2.13690018e-18+0.j, -1.12757026e-17+0.j,
           3.32526606e-22+0.j, ..., -3.46944695e-18+0.j,
           6.35274710e-22+0.j, -3.17637355e-22+0.j],
         [ 6.93889390e-18+0.j, -1.03180635e-18+0.j,
           6.35274710e-22+0.j, ..., -6.81014490e-19+0.j,
          -1.27054942e-21+0.j,  8.47032947e-22+0.j],
         [-2.48154184e-23+0.j, -1.37642854e-21+0.j,
          -2.78467610e-18+0.j, ...,  1.32348898e-22+0.j,
           2.22044605e-16+0.j,  7.39967983e-18+0.j],
         ...,
         [ 0.00000000e+00+0.j,  8.70749870e-19+0.j,
          -1.85288457e-22+0.j, ..., -2.56898650e-19+0.j,
           9.59529511e-23+0.j, -1.09187841e-22+0.j],
         [ 6.35274710e-22+0.j, -2.11758237e-22+0.j,
           0.00000000e+00+0.j, ..., -2.64697796e-23+0.j,
           7.98683392e-19+0.j, -2.16840434e-19+0.j]

In [ ]:
reload(do)
obd = do.eval_one_body_density(system, nparticles, C0)
do.plot_overlap_one_body_density(system, obd)

In [ ]:
reload(do)
dt = 1e-3
t_max = 4*2*np.pi/omega
overlap,time = do.solve_TDHF(system, dt, t_max, C0, omega, epsilon0, nparticles, animation=False)
do.plot_overlap_slater_det(system, overlap, time)

In [ ]:
a1 = np.array([[1,2,3,4],[10,11,12,13],[20,21,22,23]])
a2 = np.array([1,2,3])
a3 = a1.T*a2
print(a3.T)
np.sum(a3.T,axis=0)

In [ ]:
reload(do)
do.change_basis(system, C0, nparticles)
do.plot_MO_abc(system,potential,epsilon)